<a href="https://www.kaggle.com/code/georgelukaanya/adamnea-monthly-insights?scriptVersionId=245252477" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Load the data and set the 'created_at' column as the index
carbondioxide = pd.read_csv('/kaggle/input/ademnea-monthly-data/hive_carbondioxide_hive1.csv', index_col='created_at')
humidity = pd.read_csv('/kaggle/input/ademnea-monthly-data/hive_humidity_hive1.csv', index_col='created_at')
temperatures = pd.read_csv('/kaggle/input/ademnea-monthly-data/hive_temperatures_hive1.csv', index_col='created_at')
weights = pd.read_csv('/kaggle/input/ademnea-monthly-data/hive_weights_hive1.csv', index_col='created_at')

# Convert the index to datetime
carbondioxide.index = pd.to_datetime(carbondioxide.index)
humidity.index = pd.to_datetime(humidity.index)
temperatures.index = pd.to_datetime(temperatures.index)
weights.index = pd.to_datetime(weights.index)


# Check for and remove duplicates in the index
humidity = humidity[~humidity.index.duplicated(keep='first')]
carbondioxide = carbondioxide[~carbondioxide.index.duplicated(keep='first')]
temperatures = temperatures[~temperatures.index.duplicated(keep='first')]
weights = weights[~weights.index.duplicated(keep='first')]

In [ ]:
#Cleaning the humidity data
# Replace '*2*' with a comma in the 'record' column
humidity['record'] = humidity['record'].str.replace('*2*', ',', regex=False)

# Split the 'record' column and assign to new columns
humidity[['Interior (%)', 'Exterior (%)']] = humidity['record'].str.split(',', expand=True)

# Drop the 'record' column as it's no longer needed
humidity = humidity.drop(columns=['record'])

# Convert new columns to numeric types
humidity['Interior (%)'] = pd.to_numeric(humidity['Interior (%)'], errors='coerce')
humidity['Exterior (%)'] = pd.to_numeric(humidity['Exterior (%)'], errors='coerce')

# Replace values of 2 with NaN (without inplace to avoid warnings)
humidity['Interior (%)'] = humidity['Interior (%)'].replace(2, np.nan)
humidity['Exterior (%)'] = humidity['Exterior (%)'].replace(2, np.nan)

# Display the resulting DataFrame
humidity.head()


In [ ]:
#Cleanind the CO2 data
#replacing 2 with NaN
carbondioxide['record'] = carbondioxide.record.replace(2, np.nan)
carbondioxide.head()

In [ ]:
#Cleaning the weights data, replacing the 2 with NaN
weights.record = weights.record.replace(2, np.nan)
weights.head()

In [ ]:
#Cleaning the temperature data
# Replace '*2*' with a comma in the 'record' column
temperatures['record'] = temperatures['record'].str.replace('*2*', ',', regex=False)

# Split the 'record' column and assign to new columns
temperatures[['Interior (°C)', 'Exterior (°C)']] = temperatures['record'].str.split(',', expand=True)

# Drop the 'record' column as it's no longer needed
temperatures = temperatures.drop(columns=['record'])

# Convert new columns to numeric types
temperatures['Interior (°C)'] = pd.to_numeric(temperatures['Interior (°C)'], errors='coerce')
temperatures['Exterior (°C)'] = pd.to_numeric(temperatures['Exterior (°C)'], errors='coerce')

# Replace values of 2 with NaN (without inplace to avoid warnings)
temperatures['Interior (°C)'] = temperatures['Interior (°C)'].replace(2, np.nan)
temperatures['Exterior (°C)'] = temperatures['Exterior (°C)'].replace(2, np.nan)

# Display the resulting DataFrame
temperatures.head()


In [ ]:
# Remove NaN values from each DataFrame
carbondioxide.dropna(inplace=True)
humidity.dropna(inplace=True)
temperatures.dropna(inplace=True)
weights.dropna(inplace=True)

In [ ]:
carbondioxide.describe()

In [ ]:
weights.describe()

In [ ]:
temperatures.describe()

In [ ]:
humidity.describe()

In [ ]:
# Import the calendar module to get month names
import calendar

# Cell 1: Define the monthly_analysis function
def monthly_analysis(df, month, columns):
    """
    Perform exploratory data analysis for a given month on specified columns.
    
    Parameters:
    - df: DataFrame to analyze.
    - month: Integer month (1=January, 2=February, ...).
    - columns: List of columns for which to derive insights.
    
    Returns:
    - insights: Dictionary containing mean, standard deviation, IQR, skewness, and outliers count.
    """
    # Convert month number to month name
    month_name = calendar.month_name[month]
    
    # Filter data for the specified month
    monthly_data = df[df.index.month == month]
    
    insights = {}  # Dictionary to store calculated insights
    
    for col in columns:
        # Calculate and store each statistical metric
        insights[col] = {
            'mean': monthly_data[col].mean(),
            'std_dev': monthly_data[col].std(),
            'iqr': monthly_data[col].quantile(0.75) - monthly_data[col].quantile(0.25),
            'skewness': monthly_data[col].skew()
        }
        
        # Outlier detection based on IQR method
        q1 = monthly_data[col].quantile(0.25)
        q3 = monthly_data[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = monthly_data[(monthly_data[col] < lower_bound) | (monthly_data[col] > upper_bound)]
        
        # Remove outliers for visualization
        monthly_data_no_outliers = monthly_data[~((monthly_data[col] < lower_bound) | (monthly_data[col] > upper_bound))]
        
        # Box plot for data with outliers removed
        plt.figure(figsize=(6, 4))
        sns.boxplot(data=monthly_data_no_outliers, y=col)
        plt.title(f'{col} Box Plot ({month_name} - Outliers Removed)')
        plt.ylabel(col)
        plt.show()
        
        insights[col]['outliers_removed'] = outliers.shape[0]  # Record count of outliers
    
    return insights
    
# Function to display insights in a readable format
def display_insights(month_name, insights):
    print(f"{month_name} Insights:")
    for category, metrics in insights.items():
        print(f"\n{category}:")
        for metric, value in metrics.items():
            print(f"  {metric.capitalize()}: {value}")
    print("\n" + "="*40)


In [ ]:
# Example: Implement the analysis for April (month=4) on each dataset
co2_insights_april = monthly_analysis(carbondioxide, 4, ['record'])
humidity_insights_april = monthly_analysis(humidity, 4, ['Interior (%)', 'Exterior (%)'])
temperature_insights_april = monthly_analysis(temperatures, 4, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_april = monthly_analysis(weights, 4, ['record'])

# Displaying April insights
display_insights("April CO2", co2_insights_april)
display_insights("April Humidity", humidity_insights_april)
display_insights("April Temperature", temperature_insights_april)
display_insights("April Weight", weight_insights_april)

In [ ]:
#Perform analysis for May (month=5) on each dataset
co2_insights_may = monthly_analysis(carbondioxide, 5, ['record'])
humidity_insights_may = monthly_analysis(humidity, 5, ['Interior (%)', 'Exterior (%)'])
temperature_insights_may = monthly_analysis(temperatures, 5, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_may = monthly_analysis(weights, 5, ['record'])

# Displaying May insights
display_insights("May CO2 Insights:", co2_insights_may)
display_insights("May Humidity Insights:", humidity_insights_may)
display_insights("May Temperature Insights:", temperature_insights_may)
display_insights("May Weight Insights:", weight_insights_may)


In [ ]:
#Perform analysis for April (month=4) on each dataset
co2_insights_june = monthly_analysis(carbondioxide, 6, ['record'])
humidity_insights_june = monthly_analysis(humidity, 6, ['Interior (%)', 'Exterior (%)'])
temperature_insights_june = monthly_analysis(temperatures, 6, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_june = monthly_analysis(weights, 6, ['record'])

# Displaying April insights
display_insights("June CO2 Insights:", co2_insights_june)
display_insights("June Humidity Insights:", humidity_insights_june)
display_insights("June Temperature Insights:", temperature_insights_june)
display_insights("June Weight Insights:", weight_insights_june)


In [ ]:
#Perform analysis for June (month=6) on each dataset
co2_insights_july = monthly_analysis(carbondioxide, 7, ['record'])
humidity_insights_july = monthly_analysis(humidity, 7, ['Interior (%)', 'Exterior (%)'])
temperature_insights_july = monthly_analysis(temperatures, 7, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_july = monthly_analysis(weights, 7, ['record'])

# Displaying June insights
display_insights("July CO2 Insights:", co2_insights_july)
display_insights("July Humidity Insights:", humidity_insights_july)
display_insights("July Temperature Insights:", temperature_insights_july)
display_insights("July Weight Insights:", weight_insights_july)


In [ ]:
#Perform analysis for August (month=8) on each dataset
co2_insights_august = monthly_analysis(carbondioxide, 8, ['record'])
humidity_insights_august = monthly_analysis(humidity, 8, ['Interior (%)', 'Exterior (%)'])
temperature_insights_august = monthly_analysis(temperatures, 8, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_august = monthly_analysis(weights, 8, ['record'])

# Displaying April insights
display_insights("August CO2 Insights:", co2_insights_august)
display_insights("August Humidity Insights:", humidity_insights_august)
display_insights("August Temperature Insights:", temperature_insights_august)
display_insights("August Weight Insights:", weight_insights_august)


In [ ]:
#Perform analysis for September (month=9) on each dataset
co2_insights_september = monthly_analysis(carbondioxide, 9, ['record'])
humidity_insights_september = monthly_analysis(humidity, 9, ['Interior (%)', 'Exterior (%)'])
temperature_insights_september = monthly_analysis(temperatures, 9, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_september = monthly_analysis(weights, 9, ['record'])

# Displaying April insights
display_insights("September CO2 Insights:", co2_insights_september)
display_insights("September Humidity Insights:", humidity_insights_september)
display_insights("September Temperature Insights:", temperature_insights_september)
display_insights("September Weight Insights:", weight_insights_september)


In [ ]:
#Implement the analysis for October (month=10) on each dataset
co2_insights_october = monthly_analysis(carbondioxide, 10, ['record'])
humidity_insights_october = monthly_analysis(humidity, 10, ['Interior (%)', 'Exterior (%)'])
temperature_insights_october = monthly_analysis(temperatures, 10, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_october = monthly_analysis(weights, 10, ['record'])

# Displaying April insights
display_insights("October CO2 Insights:", co2_insights_october)
display_insights("October Humidity Insights:", humidity_insights_october)
display_insights("October Temperature Insights:", temperature_insights_october)
display_insights("October Weight Insights:", weight_insights_october)


In [ ]:
#Implement the analysis for November (month=11) on each dataset
co2_insights_november = monthly_analysis(carbondioxide, 11, ['record'])
humidity_insights_november = monthly_analysis(humidity, 11, ['Interior (%)', 'Exterior (%)'])
temperature_insights_november = monthly_analysis(temperatures, 11, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_november = monthly_analysis(weights, 11, ['record'])

# Displaying November insights
display_insights("November CO2 Insights:", co2_insights_november)
display_insights("November Humidity Insights:", humidity_insights_november)
display_insights("November Temperature Insights:", temperature_insights_november)
display_insights("November Weight Insights:", weight_insights_november)


In [ ]:
#Implement the analysis for December (month=12) on each dataset
co2_insights_december = monthly_analysis(carbondioxide, 12, ['record'])
humidity_insights_december = monthly_analysis(humidity, 12, ['Interior (%)', 'Exterior (%)'])
temperature_insights_december = monthly_analysis(temperatures, 12, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_december = monthly_analysis(weights, 12, ['record'])

# Displaying November insights
display_insights("December CO2 Insights:", co2_insights_december)
display_insights("December Humidity Insights:", humidity_insights_december)
display_insights("December Temperature Insights:", temperature_insights_december)
display_insights("December Weight Insights:", weight_insights_december)


In [ ]:
#Implement the analysis for January (month=1) on each dataset
co2_insights_january = monthly_analysis(carbondioxide, 1, ['record'])
humidity_insights_january = monthly_analysis(humidity, 1, ['Interior (%)', 'Exterior (%)'])
temperature_insights_january = monthly_analysis(temperatures, 1, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_january = monthly_analysis(weights, 1, ['record'])

# Displaying April insights
display_insights("January CO2 Insights:", co2_insights_january)
display_insights("January Humidity Insights:", humidity_insights_january)
display_insights("January Temperature Insights:", temperature_insights_january)
display_insights("January Weight Insights:", weight_insights_january)


In [ ]:
#Implement the analysis for February (month=2) on each dataset
co2_insights_february = monthly_analysis(carbondioxide, 2, ['record'])
humidity_insights_february = monthly_analysis(humidity, 2, ['Interior (%)', 'Exterior (%)'])
temperature_insights_february = monthly_analysis(temperatures, 2, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_february = monthly_analysis(weights, 2, ['record'])

# Displaying February insights
display_insights("February CO2 Insights:", co2_insights_february)
display_insights("February Humidity Insights:", humidity_insights_february)
display_insights("February Temperature Insights:", temperature_insights_february)
display_insights("February Weight Insights:", weight_insights_february)


In [ ]:
#Implement the analysis for March (month=3) on each dataset
co2_insights_december = monthly_analysis(carbondioxide, 3, ['record'])
humidity_insights_december = monthly_analysis(humidity, 3, ['Interior (%)', 'Exterior (%)'])
temperature_insights_december = monthly_analysis(temperatures, 3, ['Interior (°C)', 'Exterior (°C)'])
weight_insights_december = monthly_analysis(weights, 3, ['record'])

# Displaying November insights
display_insights("March CO2 Insights:", co2_insights_december)
display_insights("March Humidity Insights:", humidity_insights_december)
display_insights("March Temperature Insights:", temperature_insights_december)
display_insights("March Weight Insights:", weight_insights_december)


In [ ]:
# Filtered data for June only
june_data = {
    "carbondioxide": carbondioxide[carbondioxide.index.month == 6],
    "humidity": humidity[humidity.index.month == 6],
    "temperatures": temperatures[temperatures.index.month == 6],
    "weights": weights[weights.index.month == 6]
}


In [ ]:
# Internal Temperature Analysis
# 1. Calculate Uptime (number of non-NaN entries)
# 2. Calculate Average Internal Temperature for June
# 3. Plot Internal Temperature for June

internal_temp = june_data["temperatures"]['Interior (°C)']
internal_temp_uptime = internal_temp.count()
internal_temp_avg = internal_temp.mean()

print(f"June Internal Temperature Uptime: {internal_temp_uptime}")
print(f"June Internal Temperature Average: {internal_temp_avg:.2f}°C")

plt.figure(figsize=(18, 4))
sns.lineplot(data=internal_temp, label='Internal Temperature (°C)')
plt.title("June Internal Temperature Trend")
plt.ylabel("Temperature (°C)")
plt.show()


In [ ]:
# External Temperature Analysis
# 1. Calculate Uptime (number of non-NaN entries)
# 2. Calculate Average Internal Temperature for June
# 3. Plot Internal Temperature for June

external_temp = june_data["temperatures"]['Exterior (°C)']
external_temp_uptime = external_temp.count()
external_temp_avg = external_temp.mean()

print(f"June External Temperature Uptime: {external_temp_uptime}")
print(f"June External Temperature Average: {external_temp_avg:.2f}°C")

plt.figure(figsize=(18, 4))
sns.lineplot(data=external_temp, label='External Temperature (°C)')
plt.title("June External Temperature Trend")
plt.ylabel("Temperature (°C)")
plt.show()


In [ ]:
# Analyze correlation between internal and external temperature

temp_corr = internal_temp.corr(external_temp)
print(f"Correlation between June Internal and External Temperature: {temp_corr:.2f}")
sns.scatterplot(x=internal_temp, y=external_temp)
plt.title("June Internal vs. External Temperature Correlation")
plt.xlabel("Internal Temperature (°C)")
plt.ylabel("External Temperature (°C)")
plt.show()


In [ ]:
# Uptime, Average, and Plot for Internal Humidity

internal_humidity = june_data["humidity"]['Interior (%)']
internal_humidity_uptime = internal_humidity.count()
internal_humidity_avg = internal_humidity.mean()

print(f"June Internal Humidity Uptime: {internal_humidity_uptime}")
print(f"June Internal Humidity Average: {internal_humidity_avg:.2f}%")

plt.figure(figsize=(18, 4))
sns.lineplot(data=internal_humidity, label='Internal Humidity (%)')
plt.title("June Internal Humidity Trend")
plt.ylabel("Humidity (%)")
plt.show()


In [ ]:
# Uptime, Average, and Plot for External Humidity

external_humidity = june_data["humidity"]['Exterior (%)']
external_humidity_uptime = external_humidity.count()
external_humidity_avg = external_humidity.mean()

print(f"June External Humidity Uptime: {external_humidity_uptime}")
print(f"June External Humidity Average: {external_humidity_avg:.2f}%")

plt.figure(figsize=(18, 4))
sns.lineplot(data=external_humidity, label='External Humidity (%)')
plt.title("June External Humidity Trend")
plt.ylabel("Humidity (%)")
plt.show()


In [ ]:
# Correlation between Internal and External Humidity

humidity_corr = internal_humidity.corr(external_humidity)
print(f"Correlation between June Internal and External Humidity: {humidity_corr:.2f}")
sns.scatterplot(x=internal_humidity, y=external_humidity)
plt.title("June Internal vs. External Humidity Correlation")
plt.xlabel("Internal Humidity (%)")
plt.ylabel("External Humidity (%)")
plt.show()


In [1]:
# Calculate Uptime and Average CO2 levels, and plot

co2 = june_data["carbondioxide"]['record']
co2_uptime = co2.count()
co2_avg = co2.mean()

print(f"June Carbon Dioxide Uptime: {co2_uptime}")
print(f"June Carbon Dioxide Average: {co2_avg:.2f}")

plt.figure(figsize=(18, 4))
sns.lineplot(data=co2, label='Carbon Dioxide (ppm)')
plt.title("June Carbon Dioxide Trend")
plt.ylabel("CO2 (ppm)")
plt.show()


NameError: name 'june_data' is not defined

In [ ]:
# Correlation between Carbon Dioxide and Internal Temperature

co2_temp_corr = co2.corr(internal_temp)
print(f"Correlation between June Carbon Dioxide and Internal Temperature: {co2_temp_corr:.2f}")
sns.scatterplot(x=co2, y=internal_temp)
plt.title("June CO2 vs. Internal Temperature Correlation")
plt.xlabel("CO2 (ppm)")
plt.ylabel("Internal Temperature (°C)")
plt.show()


In [ ]:
# Calculate Uptime, Daily Fluctuations, Average, and plot

weight = june_data["weights"]['record']
weight_uptime = weight.count()
weight_avg = weight.mean()
weight_daily_fluctuations = weight.diff().dropna()

print(f"June Weight Uptime: {weight_uptime}")
print(f"June Weight Average: {weight_avg:.2f}")
print("June Daily Weight Fluctuations (first 5 days):")
print(weight_daily_fluctuations.head())

plt.figure(figsize=(18, 4))
sns.lineplot(data=weight, label='Weight')
plt.title("June Weight Trend")
plt.ylabel("Weight")
plt.show()


In [ ]:
# Correlation between Weight and Internal Temperature

weight_temp_corr = weight.corr(internal_temp)
print(f"Correlation between June Weight and Internal Temperature: {weight_temp_corr:.2f}")
sns.scatterplot(x=weight, y=internal_temp)
plt.title("June Weight vs. Internal Temperature Correlation")
plt.xlabel("Weight")
plt.ylabel("Internal Temperature (°C)")
plt.show()


In [ ]:
# Correlation between Weight and Carbon Dioxide
weight_co2_corr = weight.corr(co2)
print(f"Correlation between June Weight and Carbon Dioxide: {weight_co2_corr:.2f}")

# Plot weight and CO2 levels to observe any trends over time

plt.figure(figsize=(18, 6))
sns.lineplot(data=weight, label="Weight")
sns.lineplot(data=co2, label="CO2 (ppm)")
plt.title("June Weight and CO2 Comparison")
plt.xlabel("Date")
plt.legend()
plt.show()
